In [ ]:
"""
This Script parses citywide 311 noise complaints. The goal is to see what areas of the city have seen
the biggest change in noise complaints. 

TO DO:
- DONE - Download 311 data from NYC Open Data Portal (filter: 'Complaint Type' contains 'Noise')
- Create grid for data aggregation
    - What should grid size be?
    - Consider aggregating by Community District
    
- Create new DF for July 1 2019 - July 1 2020
- Create new DF for July 1 2020 - July 1 2021
- Create By year total for each year
- Create Percent change
"""

In [24]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import openpyxl


In [25]:
noise_data = r"C:\Users\MN03\Desktop\Calvin Docs\Text Amendments\Text_Amendment_Open_Restaurants\311_Noise_Jan2019_to_present\311_Noise_Jan2019_to_present_citywide.csv"

In [26]:
noise_df = pd.read_csv(noise_data)

C:\Users\MN03\miniconda3\envs\cb3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
noise_df.dtypes

Unique Key                          int64
Created Date                       object
Closed Date                        object
Agency                             object
Agency Name                        object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                      float64
Incident Address                   object
Street Name                        object
Cross Street 1                     object
Cross Street 2                     object
Intersection Street 1              object
Intersection Street 2              object
Address Type                       object
City                               object
Landmark                           object
Facility Type                      object
Status                             object
Due Date                           object
Resolution Description             object
Resolution Action Updated Date     object
Community Board                   

In [7]:
#Remove spaces from column names.replace with underscore
noise_df.columns = noise_df.columns.str.replace(' ', '_')

In [8]:
#Make column names lowercase
noise_df.columns = noise_df.columns.str.lower()

In [9]:
noise_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_(state_plane)',
       'y_coordinate_(state_plane)', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [10]:
noise_df["date_only"] = noise_df.created_date

In [11]:
#Create a column that contains only the creation date
noise_df["date_only"] = noise_df.created_date.str[:10]


In [12]:
noise_df["date_only"]

0          01/01/2019
1          01/01/2019
2          01/01/2019
3          01/01/2019
4          01/01/2019
              ...    
1748840    08/08/2021
1748841    08/08/2021
1748842    08/08/2021
1748843    08/08/2021
1748844    08/08/2021
Name: date_only, Length: 1748845, dtype: object

In [13]:
#datetime function is smart. It appears to successfully convert datetime strings without including a format string. 
noise_df["date_test"]= pd.to_datetime(noise_df["created_date"])

In [14]:
noise_df[["date_test","date_only", "created_date"]]

date_test   date_only            created_date
0       2019-01-01 00:00:29  01/01/2019  01/01/2019 12:00:29 AM
1       2019-01-01 00:00:43  01/01/2019  01/01/2019 12:00:43 AM
2       2019-01-01 00:02:13  01/01/2019  01/01/2019 12:02:13 AM
3       2019-01-01 00:03:08  01/01/2019  01/01/2019 12:03:08 AM
4       2019-01-01 00:04:26  01/01/2019  01/01/2019 12:04:26 AM
...                     ...         ...                     ...
1748840 2021-08-08 01:07:51  08/08/2021  08/08/2021 01:07:51 AM
1748841 2021-08-08 01:07:54  08/08/2021  08/08/2021 01:07:54 AM
1748842 2021-08-08 01:07:59  08/08/2021  08/08/2021 01:07:59 AM
1748843 2021-08-08 01:08:04  08/08/2021  08/08/2021 01:08:04 AM
1748844 2021-08-08 01:08:28  08/08/2021  08/08/2021 01:08:28 AM

[1748845 rows x 3 columns]

In [ ]:
"""
The code below creates, and then applies, a filter that displays all records by a specific hour.

df["messageDate"] = pd.to_datetime(df["messageDate"])
time_mask = (df['messageDate'].dt.hour >= 13) & \
            (df['messageDate'].dt.hour <= 15)
df[time_mask]
""" 
test = noise_df["date_test"].dt.hour >= 13
noise_df[test]

In [ ]:
# Create new DF that includes only complaints within a given time range. 
# The time ranges in the next cells are for different years. This will allow year over year comparison. 

date1 = '7/1/2019'
date2 = '7/1/2020'

# greater than the start date and smaller than the end date
# mask = (df['date'] > start_date) & (df['date'] <= end_date)
year1 = (noise_df["date_test"] >= date1) & \
       (noise_df["date_test"] <= date2)

In [ ]:

date3 = '7/1/2020'
date4 = '7/1/2021'

# greater than the start date and smaller than the end date
# mask = (df['date'] > start_date) & (df['date'] <= end_date)
year2 = (noise_df["date_test"] >= date3) & \
       (noise_df["date_test"] <= date4)

In [ ]:
year1noise = noise_df[year1]

In [ ]:
year2noise = noise_df[year2]

In [ ]:
"""Susan Requested checking commercial noise complaints 
from August 1, 2020 to August 1, 2021. 
1 - Create time filter for the requested dates.
2 - Create Filter for Commercial Noise complaints only.
3 - Sum by Community Board
4 - Save Results to Excel file

"""

In [43]:
excel_output = r"C:\Users\MN03\Desktop\Calvin Docs\Text Amendments\Text_Amendment_Open_Restaurants\Commercial_Noise_Complaints_Research_Outputs\commercial_noise_by_cb.xlsx"

In [15]:
# Pull records for requested year

date5 = '8/1/2020'
date6 = '8/1/2021'

year3 = (noise_df["date_test"] >= date5) & \
       (noise_df["date_test"] <= date6)

In [16]:
# Apply filter created in previous cell 

year3noise = noise_df[year3]

In [17]:
year3noise

unique_key            created_date             closed_date agency  \
891467     47051764  08/01/2020 12:00:03 AM  08/01/2020 12:23:34 AM   NYPD   
891468     47047802  08/01/2020 12:00:11 AM  08/01/2020 07:12:58 PM   NYPD   
891469     47043407  08/01/2020 12:00:15 AM  08/01/2020 02:49:51 AM   NYPD   
891470     47048666  08/01/2020 12:00:21 AM  08/01/2020 12:22:24 AM   NYPD   
891471     47042667  08/01/2020 12:00:37 AM  08/01/2020 12:46:12 AM   NYPD   
...             ...                     ...                     ...    ...   
1729761    51394726  07/31/2021 11:59:34 PM  08/01/2021 05:57:29 AM   NYPD   
1729762    51392339  07/31/2021 11:59:44 PM  08/01/2021 05:13:06 AM   NYPD   
1729763    51395344  07/31/2021 11:59:50 PM  08/01/2021 12:57:12 AM   NYPD   
1729764    51394224  07/31/2021 11:59:53 PM  08/01/2021 12:04:12 AM   NYPD   
1729765    51394970  07/31/2021 11:59:56 PM  08/01/2021 04:06:25 AM   NYPD   

                             agency_name           complaint_type  \
891467   New York City Police Department      Noise - Residential   
891468   New York City Police Department      Noise - Residential   
891469   New York City Police Department      Noise - Residential   
891470   New York City Police Department  Noise - Street/Sidewalk   
891471   New York City Police Department  Noise - Street/Sidewalk   
...                                  ...                      ...   
1729761  New York City Police Department       Noise - Commercial   
1729762  New York City Police Department      Noise - Residential   
1729763  New York City Police Department      Noise - Residential   
1729764  New York City Police Department  Noise - Street/Sidewalk   
1729765  New York City Police Department  Noise - Street/Sidewalk   

               descriptor               location_type  incident_zip  \
891467   Loud Music/Party  Residential Building/House       10468.0   
891468   Loud Music/Party  Residential Building/House       10466.0   
891469   Loud Music/Party  Residential Building/House       10456.0   
891470   Loud Music/Party             Street/Sidewalk       10027.0   
891471   Loud Music/Party             Street/Sidewalk       11101.0   
...                   ...                         ...           ...   
1729761  Loud Music/Party         Club/Bar/Restaurant       11106.0   
1729762  Loud Music/Party  Residential Building/House       11411.0   
1729763  Loud Music/Party  Residential Building/House       11355.0   
1729764  Loud Music/Party             Street/Sidewalk       10034.0   
1729765  Loud Music/Party             Street/Sidewalk       10032.0   

               incident_address  ... taxi_pick_up_location  \
891467      15 EAST  205 STREET  ...                   NaN   
891468     674 EAST  231 STREET  ...                   NaN   
891469       1169 TINTON AVENUE  ...                   NaN   
891470         55 TIEMANN PLACE  ...                   NaN   
891471          40-01 10 STREET  ...                   NaN   
...                         ...  ...                   ...   
1729761         35-55 31 STREET  ...                   NaN   
1729762       117-28 223 STREET  ...                   NaN   
1729763         42-24 149 PLACE  ...                   NaN   
1729764         107 POST AVENUE  ...                   NaN   
1729765  922 ST NICHOLAS AVENUE  ...                   NaN   

        bridge_highway_name bridge_highway_direction road_ramp  \
891467                  NaN                      NaN       NaN   
891468                  NaN                      NaN       NaN   
891469                  NaN                      NaN       NaN   
891470                  NaN                      NaN       NaN   
891471                  NaN                      NaN       NaN   
...                     ...                      ...       ...   
1729761                 NaN                      NaN       NaN   
1729762                 NaN                      NaN       NaN   
1729763                 NaN                      Na

In [19]:
# Filter out commercial noise only
cnoise = year3noise[year3noise['complaint_type'].str.contains("Commercial")]

In [33]:
# Create a sum of Commercial Noise complaints for each Community Board
# EXAMPLE: df.groupby(['Fruit','Name']).sum()
# This resets the index: df.groupby(['Fruit','Name'])['Number'].sum().reset_index()
#df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
# df.groupby(['col1', 'col2']).size().reset_index(name='counts')
cnoise_counts = cnoise.groupby('community_board').size().reset_index(name='counts')

In [34]:
cnoise_counts.columns

Index(['community_board', 'counts'], dtype='object')

In [32]:
cnoise_counts.index

Index(['0 Unspecified', '01 BRONX', '01 BROOKLYN', '01 MANHATTAN', '01 QUEENS',
       '01 STATEN ISLAND', '02 BRONX', '02 BROOKLYN', '02 MANHATTAN',
       '02 QUEENS', '02 STATEN ISLAND', '03 BRONX', '03 BROOKLYN',
       '03 MANHATTAN', '03 QUEENS', '03 STATEN ISLAND', '04 BRONX',
       '04 BROOKLYN', '04 MANHATTAN', '04 QUEENS', '05 BRONX', '05 BROOKLYN',
       '05 MANHATTAN', '05 QUEENS', '06 BRONX', '06 BROOKLYN', '06 MANHATTAN',
       '06 QUEENS', '07 BRONX', '07 BROOKLYN', '07 MANHATTAN', '07 QUEENS',
       '08 BRONX', '08 BROOKLYN', '08 MANHATTAN', '08 QUEENS', '09 BRONX',
       '09 BROOKLYN', '09 MANHATTAN', '09 QUEENS', '10 BRONX', '10 BROOKLYN',
       '10 MANHATTAN', '10 QUEENS', '11 BRONX', '11 BROOKLYN', '11 MANHATTAN',
       '11 QUEENS', '12 BRONX', '12 BROOKLYN', '12 MANHATTAN', '12 QUEENS',
       '13 BROOKLYN', '13 QUEENS', '14 BROOKLYN', '14 QUEENS', '15 BROOKLYN',
       '16 BROOKLYN', '17 BROOKLYN', '18 BROOKLYN', '26 BRONX', '27 BRONX',
       '28 BRONX', '

In [35]:
cnoise_counts

community_board  counts
0               0 Unspecified       5
1                    01 BRONX     561
2                 01 BROOKLYN    2901
3                01 MANHATTAN     629
4                   01 QUEENS    2711
..                        ...     ...
70          Unspecified BRONX      17
71       Unspecified BROOKLYN      24
72      Unspecified MANHATTAN      47
73         Unspecified QUEENS      19
74  Unspecified STATEN ISLAND       1

[75 rows x 2 columns]

In [36]:
# Display all Rows of Dataframe
# pandas.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_rows', cnoise_counts.shape[0]+1)

In [37]:
cnoise_counts

community_board  counts
0               0 Unspecified       5
1                    01 BRONX     561
2                 01 BROOKLYN    2901
3                01 MANHATTAN     629
4                   01 QUEENS    2711
5            01 STATEN ISLAND     485
6                    02 BRONX     413
7                 02 BROOKLYN     981
8                02 MANHATTAN    2078
9                   02 QUEENS     924
10           02 STATEN ISLAND     254
11                   03 BRONX     401
12                03 BROOKLYN    1168
13               03 MANHATTAN    2797
14                  03 QUEENS     944
15           03 STATEN ISLAND     237
16                   04 BRONX    1085
17                04 BROOKLYN    1450
18               04 MANHATTAN    1588
19                  04 QUEENS     721
20                   05 BRONX    1006
21                05 BROOKLYN     987
22               05 MANHATTAN     836
23                  05 QUEENS     892
24                   06 BRONX     823
25                06 BROOKLYN     888
26               06 MANHATTAN     738
27                  06 QUEENS     377
28                   07 BRONX     636
29                07 BROOKLYN     646
30               07 MANHATTAN    1106
31                  07 QUEENS     565
32                   08 BRONX     313
33                08 BROOKLYN    1124
34               08 MANHATTAN    1132
35                  08 QUEENS      79
36                   09 BRONX    1629
37                09 BROOKLYN     431
38               09 MANHATTAN     436
39                  09 QUEENS     846
40                   10 BRONX     383
41                10 BROOKLYN     972
42               10 MANHATTAN    1666
43                  10 QUEENS     450
44                   11 BRONX     361
45                11 BROOKLYN    1306
46               11 MANHATTAN     688
47                  11 QUEENS     262
48                   12 BRONX     608
49                12 BROOKLYN     298
50               12 MANHATTAN    2050
51                  12 QUEENS     453
52                13 BROOKLYN     136
53                  13 QUEENS     463
54                14 BROOKLYN     415
55                  14 QUEENS     357
56                15 BROOKLYN     362
57                16 BROOKLYN     282
58                17 BROOKLYN     491
59                18 BROOKLYN     360
60                   26 BRONX       1
61                   27 BRONX       3
62                   28 BRONX       2
63                55 BROOKLYN      18
64               64 MANHATTAN       5
65                  81 QUEENS       5
66                  82 QUEENS       1
67                  83 QUEENS       1
68                  84 QUEENS       1
69           95 STATEN ISLAND      11
70          Unspecified BRONX      17
71       Unspecified BROOKLYN      24
72      Unspecified MANHATTAN      47
73         Unspecified QUEENS      19
74  Unspecified STATEN ISLAND       1

In [47]:
# Sort values Descending
# df.sort_values(by='col1', ascending=False)
cnoise_sorted = cnoise_counts.sort_values(by='counts', ascending=False)

In [49]:
cnoise_sorted

community_board  counts
2                 01 BROOKLYN    2901
13               03 MANHATTAN    2797
4                   01 QUEENS    2711
8                02 MANHATTAN    2078
50               12 MANHATTAN    2050
42               10 MANHATTAN    1666
36                   09 BRONX    1629
18               04 MANHATTAN    1588
17                04 BROOKLYN    1450
45                11 BROOKLYN    1306
12                03 BROOKLYN    1168
34               08 MANHATTAN    1132
33                08 BROOKLYN    1124
30               07 MANHATTAN    1106
16                   04 BRONX    1085
20                   05 BRONX    1006
21                05 BROOKLYN     987
7                 02 BROOKLYN     981
41                10 BROOKLYN     972
14                  03 QUEENS     944
9                   02 QUEENS     924
23                  05 QUEENS     892
25                06 BROOKLYN     888
39                  09 QUEENS     846
22               05 MANHATTAN     836
24                   06 BRONX     823
26               06 MANHATTAN     738
19                  04 QUEENS     721
46               11 MANHATTAN     688
29                07 BROOKLYN     646
28                   07 BRONX     636
3                01 MANHATTAN     629
48                   12 BRONX     608
31                  07 QUEENS     565
1                    01 BRONX     561
58                17 BROOKLYN     491
5            01 STATEN ISLAND     485
53                  13 QUEENS     463
51                  12 QUEENS     453
43                  10 QUEENS     450
38               09 MANHATTAN     436
37                09 BROOKLYN     431
54                14 BROOKLYN     415
6                    02 BRONX     413
11                   03 BRONX     401
40                   10 BRONX     383
27                  06 QUEENS     377
56                15 BROOKLYN     362
44                   11 BRONX     361
59                18 BROOKLYN     360
55                  14 QUEENS     357
32                   08 BRONX     313
49                12 BROOKLYN     298
57                16 BROOKLYN     282
47                  11 QUEENS     262
10           02 STATEN ISLAND     254
15           03 STATEN ISLAND     237
52                13 BROOKLYN     136
35                  08 QUEENS      79
72      Unspecified MANHATTAN      47
71       Unspecified BROOKLYN      24
73         Unspecified QUEENS      19
63                55 BROOKLYN      18
70          Unspecified BRONX      17
69           95 STATEN ISLAND      11
0               0 Unspecified       5
65                  81 QUEENS       5
64               64 MANHATTAN       5
61                   27 BRONX       3
62                   28 BRONX       2
66                  82 QUEENS       1
67                  83 QUEENS       1
68                  84 QUEENS       1
60                   26 BRONX       1
74  Unspecified STATEN ISLAND       1

In [51]:
cnoise_sorted.to_excel(excel_output)

In [ ]:
"""
- Filter out CB 3 only complaints
- Filter out Commercial Noise and COVID non-compliance
- August 1 2020 - August 1 2021


"""

In [15]:
# Pull records for requested year

date5 = '8/1/2020'
date6 = '8/1/2021'

year3 = (noise_df["date_test"] >= date5) & \
       (noise_df["date_test"] <= date6)

In [16]:
# Apply filter created in previous cell 

year3noise = noise_df[year3]

In [21]:
#cb3_noise_noncompliance
# Filter out commercial noise only
cb3_noise_noncompliance = year3noise[ (year3noise['community_board'] == '03 MANHATTAN') & ((year3noise['complaint_type'].str.contains("Commercial")) | (year3noise['complaint_type'].str.contains("NonCompliance")))]



In [27]:
year3noise['complaint_type'].unique()

array(['Noise - Residential', 'Noise - Street/Sidewalk', 'Noise - Park',
       'Noise - Vehicle', 'Noise - Commercial', 'Noise',
       'Noise - Helicopter', 'Noise - House of Worship',
       'Collection Truck Noise'], dtype=object)

In [28]:
cb3_noise_noncompliance

unique_key            created_date             closed_date agency  \
891559     47048340  08/01/2020 12:08:26 AM  08/01/2020 12:34:41 AM   NYPD   
891589     47042157  08/01/2020 12:09:59 AM  08/01/2020 12:32:33 AM   NYPD   
891611     47042111  08/01/2020 12:11:09 AM  08/01/2020 12:27:06 AM   NYPD   
892477     47040240  08/01/2020 01:29:46 AM  08/01/2020 01:39:02 AM   NYPD   
893033     47062135  08/01/2020 02:24:01 AM  08/01/2020 02:26:31 AM   NYPD   
...             ...                     ...                     ...    ...   
1728999    51390127  07/31/2021 11:06:36 PM  07/31/2021 11:08:34 PM   NYPD   
1729035    51390176  07/31/2021 11:09:06 PM  07/31/2021 11:29:16 PM   NYPD   
1729207    51395131  07/31/2021 11:21:03 PM  07/31/2021 11:30:07 PM   NYPD   
1729212    51393957  07/31/2021 11:21:24 PM  07/31/2021 11:30:52 PM   NYPD   
1729514    51387526  07/31/2021 11:42:25 PM  08/01/2021 12:08:50 AM   NYPD   

                             agency_name      complaint_type  \
891559   New York City Police Department  Noise - Commercial   
891589   New York City Police Department  Noise - Commercial   
891611   New York City Police Department  Noise - Commercial   
892477   New York City Police Department  Noise - Commercial   
893033   New York City Police Department  Noise - Commercial   
...                                  ...                 ...   
1728999  New York City Police Department  Noise - Commercial   
1729035  New York City Police Department  Noise - Commercial   
1729207  New York City Police Department  Noise - Commercial   
1729212  New York City Police Department  Noise - Commercial   
1729514  New York City Police Department  Noise - Commercial   

               descriptor        location_type  incident_zip  \
891559   Loud Music/Party     Store/Commercial       10002.0   
891589   Loud Music/Party     Store/Commercial       10009.0   
891611   Loud Music/Party  Club/Bar/Restaurant       10009.0   
892477   Loud Music/Party                  NaN       10003.0   
893033   Loud Music/Party  Club/Bar/Restaurant       10003.0   
...                   ...                  ...           ...   
1728999  Loud Music/Party  Club/Bar/Restaurant       10003.0   
1729035  Loud Music/Party  Club/Bar/Restaurant       10009.0   
1729207  Loud Music/Party  Club/Bar/Restaurant       10003.0   
1729212  Loud Music/Party     Store/Commercial       10009.0   
1729514  Loud Music/Party     Store/Commercial       10002.0   

             incident_address  ... taxi_pick_up_location bridge_highway_name  \
891559       137 ESSEX STREET  ...                   NaN                 NaN   
891589            28 AVENUE B  ...                   NaN                 NaN   
891611   447 EAST   13 STREET  ...                   NaN                 NaN   
892477               3 AVENUE  ...                   NaN                 NaN   
893033   234 EAST    9 STREET  ...                   NaN                 NaN   
...                       ...  ...                   ...                 ...   
1728999  308 EAST    6 STREET  ...                   NaN                 NaN   
1729035          113 AVENUE A  ...                   NaN                 NaN   
1729207  332 EAST    9 STREET  ...                   NaN                 NaN   
1729212          112 AVENUE A  ...                   NaN                 NaN   
1729514   177 CHRYSTIE STREET  ...                   NaN                 NaN   

        bridge_highway_direction road_ramp bridge_highway_segment   latitude  \
891559                       NaN       NaN                    NaN  40.720194   
891589                       NaN       NaN                    NaN  40.722373   
891611                       NaN       NaN                    NaN  40.730215   
892477                       NaN       NaN                    NaN  40.731964   
893033                       NaN       NaN                    NaN  40.729593   
...                          ...       ...                    ...        ...   
1728999      

In [23]:
cb3_noise_noncompliance['complaint_type'].unique()

array(['Noise - Commercial'], dtype=object)

In [29]:
Output_data = r"C:\Users\MN03\Desktop\Calvin Docs\Text Amendments\Text_Amendment_Open_Restaurants\Ave_A_noise.csv"

In [30]:
# Send DF to CSV for work in GIS
#df.to_csv('out.zip', index=False,
#   compression=compression_opts)  



cb3_noise_noncompliance.to_csv(Output_data)
